In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#time tracking
import datetime

## PostGres Connections

In [3]:
import psycopg2

In [4]:
from sqlalchemy import create_engine
database='ais_test'
engine = create_engine('postgresql://patrickmaus@localhost:5432/{}'.format(database))

engine

Engine(postgresql://patrickmaus@localhost:5432/ais_test)

In [5]:
tick = datetime.datetime.now()
print('Starting processing at: ', tick)

tock = datetime.datetime.now()
lapse = tock - tick
print ('Time elapsed: {} \n'.format(lapse))

Starting processing at:  2020-03-11 08:59:47.535712
Time elapsed: 0:00:00.000424 



## Clean Port Data

In [6]:
ports_full = pd.read_csv('wpi.csv')
ports = ports_full[['index_no','port_name','latitude','longitude']]
ports = ports.rename(columns={'latitude':'lat','longitude':'lon'})
ports.head()

,index_no,port_name,lat,lon
0,61090,SHAKOTAN,43.866667,146.833333
1,61110,MOMBETSU KO,44.350000,143.350000
2,5750,CHARLOTTETOWN,46.233333,-63.133333
3,61120,ABASHIRI KO,44.016667,144.283333
4,61130,NEMURO KO,43.333333,145.583333


## Get Data

In [ ]:
port_activity = pd.read_csv('port_activity_sample.csv')
port_activity.head()
port_activity.info()

In [ ]:
ship_position = pd.read_csv('ship_position_sample.csv')
ship_position.head()
ship_position.info()

In [ ]:
df_full = pd.merge(port_activity, ship_position, how='left', on=['mmsi','time'])

In [ ]:
df_full.head()

In [ ]:
df_rick = df_full[df_full['mmsi']==538090091]
df_rick.info()

In [ ]:
df_ports = df_full[df_full['port_id'] > 0]
df_ports.info()

In [ ]:
df_full.info()

In [ ]:
print(len((df_ports['port_id'].unique())))
print(len(df_ports))


## Run DB Scan

In [7]:
from sklearn.cluster import DBSCAN

eps = .05
min_samples = 50

X = df_rick[['lon', 'lat']].values

NameError: name 'df_rick' is not defined

In [ ]:
tick = datetime.datetime.now()
print('Starting processing with eps={} and min_samples={} at: '.format(str(eps), str(min_samples)), tick)


dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X)

print('Number of unique labels: ', np.unique(dbscan.labels_))
print('Number of  Core Samples:' , len(dbscan.core_sample_indices_))

results_dict = {'clust_id': dbscan.labels_,'lat':X[:, 0],'lon':X[:,1]}
df_results = pd.DataFrame(results_dict)

tock = datetime.datetime.now()
lapse = tock - tick
print ('Time elapsed: {} \n'.format(lapse))


## Find Center of Each Cluster and compare to nearest Port

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956  # 6371 Radius of earth in kilometers. Use 3956 for miles
    return c * r

def determine_min_distances(df1, name_1, df2, name_2):
    min_distances = []
    for i in range(len(df1)):
        lon1 = df1['lon'].loc[i]
        lat1 = df1['lat'].loc[i]
        distances = []
        for x in range(len(df2)):
            lon2 = df2['lon'].loc[x]
            lat2 = df2['lat'].loc[x]
            dist = haversine(lon1, lat1, lon2, lat2)
            distances.append((round(dist,3),df1[name_1].loc[i],df2[name_2].loc[x]))
        min_distances.append(min(distances))
    return(min_distances)

In [ ]:
dist = determine_min_distances(centers,'clust_id',ports,'port_name')
df_dist = pd.DataFrame(dist, columns=['distance from center', 'clust_id', 'nearest_port'])

In [ ]:
# group the results from the haversine by mean to get the centerpoint of the cluster
centers = df_results.groupby('clust_id').mean().reset_index()
# group the same results by count to get the total number of positions
counts = df_results.groupby('clust_id').count()
# select only one column, in this case I chose lat
counts['counts'] = counts['lat']
# drop the other columns so count is now just the clust_id and the summed counts
counts.drop(['lat','lon'], axis=1, inplace=True)
# merge counts and centers
centers = pd.merge(centers, counts, how='left', on='clust_id')
# merge the full centers file with the results of the haversine equation
centers = pd.merge(centers, df_dist, how='left', on='clust_id')
centers.head()


## Adding to Database for QGIS visualization

In [ ]:
#%% Make and test conn and cursor
conn = psycopg2.connect(host="localhost",database=database)
c = conn.cursor()
if c:
    print('Connection to {} is good.'.format(database))
else:
    print('Error connecting.')
c.close()

In [ ]:
def df_to_table_with_geom(df, name, eps, min_samples):
    # add the eps and min_samples value to table name
    new_table_name = 'dbscan_results_' + name + '_' + '_' + str(min_samples)
    
    # drop table if an old one exists
    c = conn.cursor()
    c.execute("""DROP TABLE IF EXISTS {}""".format(new_table_name))
    conn.commit()
    c.close()
    # make a new table with the df
    df.to_sql(new_table_name, engine)
    # add a geom column to the new table and populate it from the lat and lon columns
    c = conn.cursor()
    c.execute("""ALTER TABLE {} ADD COLUMN 
                geom geometry(Point, 4326);""".format(new_table_name))
    conn.commit()
    c.execute("""UPDATE {} SET 
                geom = ST_SetSRID(ST_MakePoint(lon, lat), 4326);""".format(new_table_name))
    conn.commit()
    c.close()

In [ ]:
df_to_table_with_geom(df_results, 'df_rick', eps, min_samples)